In [1]:
import h5py
import rebound
import numpy as np
import pandas as pd
import dask.dataframe as dd
from celmech import Andoyer
from spock import FeatureClassifier
import mr_forecast as mr

In [2]:
dfC = pd.read_csv("Kepler-23_default_priors_SPOCK.csv", index_col=0)
dfC.head()

,sim,k,h,Z12,Zcom12,Z23,Zcom23,e1,e2,e3,m1,m2,m3,probstability,m1+m2,m2+m3,total m
0,0.0,-0.193662,-0.285635,0.388702,0.636611,0.686698,0.664992,0.867296,0.345097,0.712553,6.213249,13.376343,40.220045,0.0,19.589593,53.596389,59.809638
1,1.0,0.342728,-0.343074,0.332209,0.559205,0.597740,0.191515,0.357091,0.484935,0.377275,0.485521,4.204200,3.251008,0.0,4.689721,7.455208,7.940729
2,2.0,0.210379,-0.115803,0.217567,0.243371,0.598793,0.408302,0.269689,0.240145,0.559020,1.452781,4.575729,11.466701,0.0,6.028510,16.042429,17.495211
3,3.0,0.024894,0.395947,0.292038,0.090441,0.240557,0.312442,0.046320,0.396729,0.026889,19.684153,3.179315,1.719421,0.0,22.863469,4.898737,24.582890
4,4.0,-0.379218,0.096536,0.209718,0.486905,0.836410,0.423220,0.227684,0.391312,0.701445,0.858439,159.849724,0.681329,0.0,160.708163,160.531053,161.389493


In [3]:
dfC.shape

(27053, 17)

# Used Mstar = 1 +/- 0.1. Huber table (see van Eylen & Albrecht 2015) has 1.07. All that matters is mu, Mstar just changes Porb, so we run slightly more or less than 1e9. Much smaller than other uncertainties,  so we ignore

Seems that Christian's avars23 used a10 = sim.particles[1].a leading to different values for especially Z23. Masses match exactly as expected, but we want to record mu since that is what matters. We also record Mstar to be able to translate back and convert orbits to time.

In [5]:
def init_process():
    global model
    model = FeatureClassifier()
    
def populate(row): # Assumes inner two near 3:2 and outer two near 7:5 like in Kepler23
    safilename = 'K23uniform/K23_logm_sa_{0}.bin'.format(int(row['sim']))
    sa = rebound.SimulationArchive(safilename)
    sim = sa[0]
    P1 = sim.particles[1].P
    
    avars12 = Andoyer.from_Simulation(sim, j=3, k=1, i1=1, i2=2)
    row['Mstar'] = sim.particles[0].m
    row['Z12'] = avars12.Z
    row['Zcom12'] = avars12.Zcom
    avars23 = Andoyer.from_Simulation(sim, j=7, k=2, i1=2, i2=3)
    row['Z23'] = avars23.Z
    row['Zcom23'] = avars23.Zcom
    row['e1'] = sim.particles[1].e
    row['e2'] = sim.particles[2].e
    row['e3'] = sim.particles[3].e
    row['h1'] = sim.particles[1].e*np.sin(sim.particles[1].pomega)
    row['h2'] = sim.particles[2].e*np.sin(sim.particles[2].pomega)
    row['h3'] = sim.particles[3].e*np.sin(sim.particles[3].pomega)
    row['k1'] = sim.particles[1].e*np.cos(sim.particles[1].pomega)
    row['k2'] = sim.particles[2].e*np.cos(sim.particles[2].pomega)
    row['k3'] = sim.particles[3].e*np.cos(sim.particles[3].pomega)
    row['mu1'] = sim.particles[1].m/sim.particles[0].m
    row['mu2'] = sim.particles[2].m/sim.particles[0].m
    row['mu3'] = sim.particles[3].m/sim.particles[0].m
    row['mu1+mu2'] = row['mu1'] + row['mu2']
    row['mu2+mu3'] = row['mu2'] + row['mu3']
    row['mutot'] = row['mu1'] + row['mu2'] + row['mu3']
    hcom = (row['mu1']*row['h1'] + row['mu2']*row['h2'] + row['mu3']*row['h3'])/(row['mu1']+row['mu2']+row['mu3'])
    kcom = (row['mu1']*row['k1'] + row['mu2']*row['k2'] + row['mu3']*row['k3'])/(row['mu1']+row['mu2']+row['mu3'])
    row['ecom'] = np.sqrt(hcom**2 + kcom**2)
    
    simf = sa[-1]
    row['t_instability'] = simf.t/P1 # in # orbits
    row['nbodystable'] = row['t_instability'] > 8.99e8
    return row

def spockpred(ID):
    safilename = 'K23uniform/K23_logm_sa_{0}.bin'.format(int(ID))
    sa = rebound.SimulationArchive(safilename)
    sim = sa[0]
    return model.predict_stable(sim)

In [6]:
%%time
import dask.dataframe as dd
ddf = dd.from_pandas(df, npartitions=40)
df = ddf.apply(populate, axis=1, meta=pd.DataFrame([populate(df.loc[0])])).compute(scheduler='processes')

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/work

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/wor

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/work

CPU times: user 707 ms, sys: 558 ms, total: 1.26 s
Wall time: 46.7 s


In [7]:
from multiprocessing import Pool
pool = Pool(processes=48, initializer=init_process)

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/work

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/home/dtamayo/miniconda3/envs/ml/li

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/wor

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/work

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/wor

/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
/home/dtamayo/miniconda3/envs/ml/lib/python3.7/site-packages/numpy/lib/function_base.py:3405: RuntimeWarning: Invalid value encountered in median
  r 

In [8]:
%%time
simIDs = df['sim'].values
df['spockprob'] = pool.map(spockpred, simIDs)

CPU times: user 1.7 s, sys: 987 ms, total: 2.68 s
Wall time: 18min 53s


In [16]:
df.tail()

,sim,Mstar,Z12,Zcom12,Z23,Zcom23,e1,e2,e3,h1,...,mu1,mu2,mu3,mu1+mu2,mu2+mu3,mutot,ecom,t_instability,nbodystable,spockprob
27048,1999380.0,0.880613,0.201449,0.139535,0.102941,0.132488,0.198727,0.155031,0.014555,-0.099066,...,2.729311e-06,0.000012,0.000003,0.000015,0.000015,0.000018,0.101288,5.358731e+04,0.0,0.010296
27049,1999466.0,1.214257,0.028842,0.012843,0.021864,0.008439,0.039781,0.010983,0.022669,-0.039776,...,8.367568e-07,0.000042,0.000007,0.000043,0.000049,0.000050,0.008076,1.000000e+09,1.0,0.787886
27050,1999693.0,1.225737,0.075410,0.120734,0.066761,0.111895,0.088874,0.108052,0.175515,0.080759,...,9.338099e-06,0.000078,0.000004,0.000087,0.000081,0.000091,0.104393,6.319852e+04,0.0,0.009134
27051,1999943.0,1.083352,0.130139,0.262373,0.077109,0.180838,0.224166,0.236365,0.140135,0.222827,...,1.474248e-06,0.000006,0.000018,0.000007,0.000024,0.000025,0.156694,2.133985e+04,0.0,0.005355
27052,1999969.0,1.159434,0.046016,0.032291,0.051997,0.031372,0.095140,0.020418,0.050637,-0.057283,...,3.549245e-06,0.000037,0.000085,0.000040,0.000122,0.000125,0.026185,5.152893e+05,0.0,0.014513


In [10]:
df.to_csv('Kepler-23_stability.csv')

In [5]:
import pandas as pd
df = pd.read_csv('Kepler-23_stability.csv', index_col=0)
df['t_instability'].sum()/1e9*3/48/24

3.4748793602640635

# van Eylen 2015 figure sets are different in the arxiv and apj versions. arxiv matches folder sent by vincent. Note that the numbers are in different order than planets

* 03 = b
* 01 = c
* 02 = d
# omega measured CCW from line of sight at which planet transits. rows in ecc list and w list match up in order 

In [19]:
eb = np.loadtxt('Kepler23vanEylen15/eccentricity_listK00168.03.txt')
wb = np.loadtxt('Kepler23vanEylen15/omega_listK00168.03.txt')
ec = np.loadtxt('Kepler23vanEylen15/eccentricity_listK00168.01.txt')
wc = np.loadtxt('Kepler23vanEylen15/omega_listK00168.01.txt')
ed = np.loadtxt('Kepler23vanEylen15/eccentricity_listK00168.02.txt')
wd = np.loadtxt('Kepler23vanEylen15/omega_listK00168.02.txt')

In [20]:
Mearth = 3e-6
yr = 365.25 # days
Rsun = 1/215 # AU
Mstar = 1.078
Rstar = 1.548*Rsun
Pb = 7.107
Pc = 10.742
Pd = 15.274
ad = (Pd/yr)**(2/3)*(Mstar)**(1/3)

# Radii from DR 25 (see exomasses/eccentricities.ipynb)
Rb = 1.72 # +0.11 - 0.07 Rearth
Rc = 3.06 # +0.19 - 0.12 Rearth
Rd = 2.17 # +0.13 - 0.09 Rearth

imax = Rstar/ad # max inclination for outermost planet to transit
print(imax*180/np.pi)

def vanEylensim(ID):
    np.random.seed(ID)
    sim = rebound.Simulation()
    sim.units = ['days', 'AU', 'Msun']
    sim.add(m=Mstar)
    sim.add(m=mr.Rpost2M([Rb], unit='Earth', classify='No')[0]*Mearth, P=Pb, M=np.random.random()*2*np.pi, e=np.abs(eb[ID]), pomega=wb[ID], inc=np.random.random()*imax, Omega=np.random.random()*2*np.pi)
    sim.add(m=mr.Rpost2M([Rc], unit='Earth', classify='No')[0]*Mearth, P=Pc, M=np.random.random()*2*np.pi, e=np.abs(ec[ID]), pomega=wc[ID], inc=np.random.random()*imax, Omega=np.random.random()*2*np.pi)
    sim.add(m=mr.Rpost2M([Rd], unit='Earth', classify='No')[0]*Mearth, P=Pd, M=np.random.random()*2*np.pi, e=np.abs(ed[ID]), pomega=wd[ID], inc=np.random.random()*imax, Omega=np.random.random()*2*np.pi)
    return sim

def vanEylenrow(row): # Assumes inner two near 3:2 and outer two near 7:5 like in Kepler23
    sim = vanEylensim(row['ID'])
    P1 = sim.particles[1].P
    
    avars12 = Andoyer.from_Simulation(sim, j=3, k=1, i1=1, i2=2)
    row['Mstar'] = 1.078
    row['Z12'] = avars12.Z
    row['Zcom12'] = avars12.Zcom
    avars23 = Andoyer.from_Simulation(sim, j=7, k=2, i1=2, i2=3)
    row['Z23'] = avars23.Z
    row['Zcom23'] = avars23.Zcom
    row['e1'] = sim.particles[1].e
    row['e2'] = sim.particles[2].e
    row['e3'] = sim.particles[3].e
    row['mu1'] = sim.particles[1].m/sim.particles[0].m
    row['mu2'] = sim.particles[2].m/sim.particles[0].m
    row['mu3'] = sim.particles[3].m/sim.particles[0].m
    row['mu1+mu2'] = row['mu1'] + row['mu2']
    row['mu2+mu3'] = row['mu2'] + row['mu3']
    row['mutot'] = row['mu1'] + row['mu2'] + row['mu3']
    row['h1'] = sim.particles[1].e*np.sin(sim.particles[1].pomega)
    row['h2'] = sim.particles[2].e*np.sin(sim.particles[2].pomega)
    row['h3'] = sim.particles[3].e*np.sin(sim.particles[3].pomega)
    row['k1'] = sim.particles[1].e*np.cos(sim.particles[1].pomega)
    row['k2'] = sim.particles[2].e*np.cos(sim.particles[2].pomega)
    row['k3'] = sim.particles[3].e*np.cos(sim.particles[3].pomega)
    hcom = (row['mu1']*row['h1'] + row['mu2']*row['h2'] + row['mu3']*row['h3'])/(row['mu1']+row['mu2']+row['mu3'])
    kcom = (row['mu1']*row['k1'] + row['mu2']*row['k2'] + row['mu3']*row['k3'])/(row['mu1']+row['mu2']+row['mu3'])
    row['ecom'] = np.sqrt(hcom**2 + kcom**2)
    
    return row

3.3394414552197382


In [21]:
dfvanEylen = pd.DataFrame()
dfvanEylen['ID'] = range(40000)

In [22]:
%%time
dfvanEylen = dfvanEylen.apply(vanEylenrow, axis=1)

CPU times: user 20min 17s, sys: 49.5 s, total: 21min 6s
Wall time: 20min 22s


In [23]:
dfvanEylen.to_csv('Kepler-23_transitdurations.csv')

# TTVs

In [17]:
%%time
try:
    df = pd.read_csv("Kepler-23_TTVs.csv", index_col=0)
except:
    post_file = "NBody_MCMC_Posteriors.hdf5"
    post_data = h5py.File(post_file,"r")
    data = post_data['Kepler-23/DefaultPriors/PosteriorSample']
    npl = data.shape[1]//5
    col_heads = []
    for i in range(npl):
        col_heads += ['mu{}'.format(i+1),'per{}'.format(i+1),'h{}'.format(i+1),'k{}'.format(i+1),'Tc{}'.format(i+1)]
    df = pd.DataFrame(data=data,columns=col_heads)
    print(df.shape)
    
    for i in range(npl):
        h = df['h{}'.format(i+1)]
        k = df['k{}'.format(i+1)]
        per = df['per{}'.format(i+1)]
        Tc = df['Tc{}'.format(i+1)]
        df['pomega{}'.format(i+1)] = np.arctan2(k,h)
        df['e{}'.format(i+1)] = np.sqrt(h**2 + k**2)
        df['mean_long{}'.format(i+1)] = np.mod(-2 * np.pi * Tc / per,2*np.pi)
    hcom = (df['mu1']*df['h1'] + df['mu2']*df['h2'] + df['mu3']*df['h3'])/(df['mu1']+df['mu2']+df['mu3'])
    kcom = (df['mu1']*df['k1'] + df['mu2']*df['k2'] + df['mu3']*df['k3'])/(df['mu1']+df['mu2']+df['mu3'])
    df['ecom'] = np.sqrt(hcom**2 + kcom**2)
    
    df.to_csv("Kepler-23_TTVs.csv")

(250000, 15)
CPU times: user 53.2 s, sys: 738 ms, total: 53.9 s
Wall time: 47.4 s


In [18]:
def addZ(row):
    sim = rebound.Simulation()
    sim.units = ('Msun','days','AU')
    sim.add(m=1)
    for i in range(npl):
        mu = row['mu{}'.format(i+1)]
        per = row['per{}'.format(i+1)]
        e = row['e{}'.format(i+1)]
        mean_long = row['mean_long{}'.format(i+1)]
        pomega = row['pomega{}'.format(i+1)]
        sim.add(m=mu,P=per,e=e,l=mean_long,pomega=pomega,hash=i)
    sim.move_to_com()
    
    row['Mstar'] = 1.078
    avars12 = Andoyer.from_Simulation(sim, j=3, k=1, i1=1, i2=2)
    row['Z12'] = avars12.Z
    row['Zcom12'] = avars12.Zcom
    avars23 = Andoyer.from_Simulation(sim, j=7, k=2, i1=2, i2=3)
    row['Z23'] = avars23.Z
    row['Zcom23'] = avars23.Zcom
    
    return row

In [19]:
%%time
df = df.apply(addZ, axis=1)

CPU times: user 51min 43s, sys: 53.9 s, total: 52min 37s
Wall time: 51min 51s


In [25]:
df.tail()

,mu1,per1,h1,k1,Tc1,mu2,per2,h2,k2,Tc2,...,mean_long2,pomega3,e3,mean_long3,ecom,Mstar,Z12,Zcom12,Z23,Zcom23
249995,0.000003,7.105870,-0.038511,-0.196506,57.031001,0.000006,10.743525,0.032910,-0.196457,55.659821,...,5.147323,-1.075444,0.199238,4.333169,0.195363,1.078,0.057631,0.225597,0.052884,0.202704
249996,0.000003,7.106110,-0.069445,0.025974,56.975209,0.000006,10.743671,0.000208,-0.003399,55.525812,...,5.226137,-0.406624,0.062597,4.431526,0.002368,1.078,0.048022,0.029157,0.047269,0.025118
249997,0.000004,7.106614,-0.001841,-0.075112,57.111239,0.000004,10.743291,0.044821,-0.081911,55.689775,...,5.129095,-0.580675,0.077984,4.414196,0.075583,1.078,0.039443,0.087032,0.029061,0.090067
249998,0.000003,7.106508,-0.087985,0.084049,56.927508,0.000004,10.743239,-0.016223,0.039809,55.475669,...,5.254157,0.894684,0.026970,4.517402,0.044281,1.078,0.048540,0.084217,0.025173,0.031784
249999,0.000004,7.106154,-0.122730,-0.000055,56.853333,0.000009,10.743234,-0.046245,-0.023290,55.364239,...,5.319310,-2.470655,0.004345,4.557613,0.066148,1.078,0.045531,0.083131,0.029906,0.047402


In [27]:
df.to_csv("Kepler-23_TTVs.csv")